In [0]:
# 01_ingest_bronze
from pyspark.sql import functions as F

# 2) Global temp views lesen (WICHTIG: global_temp.<name>)
customers = spark.table("global_temp.customers_bronze")
orders = spark.table("global_temp.orders_bronze")
items = spark.table("global_temp.order_items_bronze")

# 3) Ingest-Metadaten hinzufügen
ingest_ts = F.current_timestamp()
customers_b = customers.withColumn("ingest_ts", ingest_ts)
orders_b = orders.withColumn("ingest_ts", ingest_ts)
items_b = items.withColumn("ingest_ts", ingest_ts)

# 4) Sanity checks (Actions)
print("customers rows:", customers_b.count())
print("orders rows:", orders_b.count())
print("order_items rows:", items_b.count())

# 5) Wieder als GLOBAL temp views bereitstellen -> damit sie von anderen Notebooks gefunden werden können
customers_b.createOrReplaceGlobalTempView("customers_bronze_v")
orders_b.createOrReplaceGlobalTempView("orders_bronze_v")
items_b.createOrReplaceGlobalTempView("order_items_bronze_v")

# 6) Sichtprobe
display(customers_b)
display(orders_b)
display(items_b)